In [8]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import regularizers
# load ascii text and covert to lowercase
filename = "data.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'), kernel_regularizer=regularizers.l1(0.001))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Total Characters:  833
Total Vocab:  27
Total Patterns:  733
Epoch 1/20
733/733 [==============================] - 9s 12ms/step - loss: 3.2525

Epoch 00001: loss improved from inf to 3.25250, saving model to weights-improvement-01-3.2525.hdf5
Epoch 2/20
733/733 [==============================] - 6s 8ms/step - loss: 2.9577

Epoch 00002: loss improved from 3.25250 to 2.95767, saving model to weights-improvement-02-2.9577.hdf5
Epoch 3/20
733/733 [==============================] - 6s 8ms/step - loss: 2.8726

Epoch 00003: loss improved from 2.95767 to 2.87256, saving model to weights-improvement-03-2.8726.hdf5
Epoch 4/20
733/733 [==============================] - 6s 8ms/step - loss: 2.8657

Epoch 00004: loss improved from 2.87256 to 2.86573, saving model to weights-improvement-04-2.8657.hdf5
Epoch 5/20
733/733 [==============================] - 6s 8ms/step - loss: 2.8447

Epoch 00005: loss improved from 2.86573 to 2.84469, saving model to weights-improvement-05-2.8447.hdf5
Epoch 6/20
733/73

In [ ]:
# pick a random seed
import sys
int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")